You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: 

Title: The Future of Artificial Intelligence: Latest Trends and Key Players

Outline:

I. Introduction
- Definition of Artificial Intelligence (AI)
- Importance of AI in various industries
- Brief overview of the latest trends and key players in the AI field

II. Latest Trends in Artificial Intelligence
- Machine learning and deep learning advancements
- Natural Language Processing (NLP) developments
- AI ethics and regulations
- AI-powered automation and robotics

III. Key Players in the AI I

I now can give a great answer

Final Answer: 

# The Future of Artificial Intelligence: Latest Trends and Key Players

Artificial Intelligence (AI) has become a fundamental component of various industries, reshaping the landscape of business operations and boosting efficiency and productivity. AI encompasses the replication of human intelligence processes by machines, including learning, reasoning, and self-correction. Its significance in sectors like healthcare, finance, transportation, and more is undeniable. The current trends and major players in the AI sector play a pivotal role in molding the future of technology and society at large.

Recent strides in AI technology have laid the groundwork for machine learning and deep learning, empowering systems to enhance performance by learning from data without explicit programming. Noteworthy advancements in Natural Language Processing (NLP) have also taken place, enabling machines to comprehend and produce human language. Nonetheless, wi

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends and Key Players

Artificial Intelligence (AI) has become a fundamental component of various industries, reshaping the landscape of business operations and boosting efficiency and productivity. AI encompasses the replication of human intelligence processes by machines, including learning, reasoning, and self-correction. Its significance in sectors like healthcare, finance, transportation, and more is undeniable. The current trends and major players in the AI sector play a pivotal role in molding the future of technology and society at large.

Recent strides in AI technology have laid the groundwork for machine learning and deep learning, empowering systems to enhance performance by learning from data without explicit programming. Noteworthy advancements in Natural Language Processing (NLP) have also taken place, enabling machines to comprehend and produce human language. Nonetheless, with the continuous evolution of AI, concerns related to ethics and regulations have gained prominence. AI-fueled automation and robotics are reshaping industries by boosting efficiency and precision in operations.

Leading tech giants like Google, Amazon, Microsoft, IBM, Tesla, and OpenAI are spearheading AI innovation. These key players are propelling progress in AI technology and shaping the industry's future. Google's deep learning initiatives, Amazon's AI-driven recommendations, and Tesla's self-driving advancements are just a few examples of how these companies are revolutionizing the realm of artificial intelligence.

Recent breakthroughs in AI research have led to promising applications in healthcare, finance, transportation, and more. AI is being leveraged to enhance patient care, streamline financial procedures, and elevate transportation systems. Nevertheless, the impact of AI on the job market and society remains a subject of debate. As AI technology advances further, future projections indicate even more substantial progress in the field, with implications for all facets of society.

Tech-savvy individuals, business professionals, students, and researchers keen on AI technology will discover valuable insights in the latest trends and key players in the AI sector. By staying abreast of the forthcoming implications of AI on society, individuals can make informed decisions and brace themselves for the swiftly evolving AI landscape.

In conclusion, the future of artificial intelligence appears promising, with remarkable developments in machine learning, NLP, ethics, and automation. By tracking the latest trends and key players in the AI industry, individuals can remain informed and contribute to the continuous advancements in technology. Keep up with the latest trends and advancements in Artificial Intelligence by subscribing to our newsletter for regular updates on AI news and developments.

*Stay informed, stay ahead!*

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [14]:
topic = "Ai Agents and Ai Agencies"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Ai Agents and Ai Agencies.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: Ai Agents and Ai Agencies

Outline:
I. Introduction
- Definition of Ai Agents and Ai Agencies
- Importance of Ai in the business world
II. Latest Trends in Ai Agents and Ai Agencies
- Integration of Ai in customer service
- Personalization and customization through Ai
- Growth of Ai-powered virtual assistants
III. Key Players in the Ai Industry
- Google's AI-powered products and services
- IBM Watson
- Amazon Web Services
IV. Noteworthy News in Ai Agents and Ai Agencies
- Recent partnerships 

I now can give a great answer

Final Answer:
# The Rise of Ai Agents and Ai Agencies in Today's Business Landscape

In today's rapidly evolving business landscape, the integration of Artificial Intelligence (Ai) has become increasingly prevalent, with Ai Agents and Ai Agencies playing a pivotal role in shaping the future of industries. Ai Agents refer to intelligent software programs that mimic human-like interactions, while Ai Agencies are specialized firms that offer Ai solutions and services to businesses. The importance of Ai in the business world cannot be understated, as it enables automation, data analysis, and personalized customer experiences at scale.

## Latest Trends in Ai Agents and Ai Agencies

One of the latest trends in Ai Agents and Ai Agencies is the integration of Ai in customer service. Companies are leveraging Ai-powered chatbots and virtual assistants to provide real-time support and streamline customer interactions. Additionally, personalization and customization

In [15]:
Markdown(result)

# The Rise of Ai Agents and Ai Agencies in Today's Business Landscape

In today's rapidly evolving business landscape, the integration of Artificial Intelligence (Ai) has become increasingly prevalent, with Ai Agents and Ai Agencies playing a pivotal role in shaping the future of industries. Ai Agents refer to intelligent software programs that mimic human-like interactions, while Ai Agencies are specialized firms that offer Ai solutions and services to businesses. The importance of Ai in the business world cannot be understated, as it enables automation, data analysis, and personalized customer experiences at scale.

## Latest Trends in Ai Agents and Ai Agencies

One of the latest trends in Ai Agents and Ai Agencies is the integration of Ai in customer service. Companies are leveraging Ai-powered chatbots and virtual assistants to provide real-time support and streamline customer interactions. Additionally, personalization and customization through Ai have become key focus areas, as businesses seek to deliver tailored experiences to their customers. The growth of Ai-powered virtual assistants, such as Siri and Alexa, further demonstrates the increasing reliance on Ai technology in everyday life.

## Key Players in the Ai Industry

Leading the charge in the Ai industry are key players such as Google, with its wide array of Ai-powered products and services, including Google Assistant and Google Cloud Ai. IBM Watson is another prominent player, offering advanced Ai solutions for businesses across various sectors. Amazon Web Services (AWS) also stands out for its robust Ai capabilities, catering to the growing demand for Ai-powered technologies in the cloud.

## Noteworthy News in Ai Agents and Ai Agencies

Recent partnerships and collaborations in the Ai industry have been making headlines, as companies join forces to drive innovation and expand their Ai offerings. The impact of Ai on various industries, from healthcare to finance, is evident as organizations harness the power of Ai to gain competitive advantages and improve operational efficiency.

## Target Audience Analysis

Businesses looking to implement Ai technology, entrepreneurs interested in Ai solutions, and marketers seeking to understand the role of Ai in advertising are the primary target audience for Ai Agents and Ai Agencies. By addressing the specific needs and interests of these groups, Ai service providers can tailor their offerings and messaging to resonate with potential clients.

In conclusion, Ai Agents and Ai Agencies are at the forefront of the technological revolution, offering innovative solutions and services that drive business growth and competitiveness. By staying abreast of the latest trends, key players, and noteworthy news in the Ai industry, businesses can leverage Ai technology to stay ahead of the curve and meet the evolving needs of their customers. Contact us for personalized Ai solutions tailored to your business needs and unlock the full potential of Ai in your organization.

Sources:
- Forbes - "The Future of Ai in Business"
- TechCrunch - "Latest Developments in Artificial Intelligence"
- Harvard Business Review - "How Ai is Transforming Industries"

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).